# JUICE RPWI HF SID6 (PSSR2 surv): L1a QL -- 2023/10/29

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_math_lib as juice_math
import juice_sid6_lib as juice_sid6

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
unit_mode = 0                           # 0: sum    1: /Hz
cal_mode = 0                            # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
cf, p_max, p_min = juice_cdf.cal_factors(unit_mode, cal_mode, cal, p_raw_max, p_raw_min)

In [ ]:
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder
data_dir = '/Users/user/0-python/JUICE_data/'        # CDF data folder

# *** Ver.2 ***

# 202310 -- SAMPLE
data_name = 'test-CCSDS/sample/cdf/SID06_20231024-0049.cdf'

# get CDF data

In [ ]:
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid6.juice_getdata_hf_sid6(cdf)
date = data.epoch[0]
month = date.strftime('%Y%m')
print(data.epoch[0], "-", data.epoch[-1], "[", month, "]")

n_time = data.auto_corr.shape[0]
n_time0 = n_time-1
print("data size:", data.auto_corr.shape)  # n_time, 16, data.N_samp_AUX[0]

data.auto_corr = data.auto_corr * 10**(cf/10)

In [ ]:
# Mode 
print("==> Ch:", data.ch_selected[0], "  Num-sweep:", n_time, "   Num-AutoCorr:", data.N_step_AUX[0], "   each Length:", data.N_auto_corr[0] )

# Raw data

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax1 = fig.add_subplot(4, 1, 1)
ax2 = fig.add_subplot(4, 1, 2)

ax1.plot(np.ravel(data.auto_corr[:][:]), linewidth=.5)
ax2.plot(np.ravel(data.epoch[:]), '.')
ax1.set_ylabel('AutoCorr [RAW @ ADC]')
date = data.epoch[0]
date = date.strftime('%Y/%m/%d')
ax2.set_xlabel(date)

"""
# range: X-axis
xlim=[0, N_frequency-1]
ax1.set_xlim(xlim)
ax2.set_xlim(xlim)
ax3.set_xlim(xlim)
# range: Y-axis
ylim=[10**p_raw_min, 10**p_raw_max]
ax1.set_ylim(ylim)
ax2.set_ylim(ylim)
ylim=[f_min, f_max]
ax3.set_ylim(ylim)
"""

# Plot
fig.show
png_fname = work_dir+data_name+'_raw.png'
# fig.savefig(png_fname)

# First data

In [ ]:
n_sweep = 0

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 4, 1)
ax2 = fig.add_subplot(4, 4, 2)
ax3 = fig.add_subplot(4, 4, 3)
ax4 = fig.add_subplot(4, 4, 4)
ax5 = fig.add_subplot(4, 4, 5)
ax6 = fig.add_subplot(4, 4, 6)
ax7 = fig.add_subplot(4, 4, 7)
ax8 = fig.add_subplot(4, 4, 8)
ax9 = fig.add_subplot(4, 4, 9)
ax10 = fig.add_subplot(4, 4, 10)
ax11 = fig.add_subplot(4, 4, 11)
ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13)
ax14 = fig.add_subplot(4, 4, 14)
ax15 = fig.add_subplot(4, 4, 15)
ax16 = fig.add_subplot(4, 4, 16)

ax1.plot(data.time, data.auto_corr[n_sweep][0][:], '-r', linewidth=.5)
ax1.plot(data.time, data.auto_corr[n_sweep][1][:], '-g', linewidth=.5)
ax1.plot(data.time, data.auto_corr[n_sweep][2][:], '-b', linewidth=.5)
ax2.plot(data.time, data.auto_corr[n_sweep][3][:], '-r', linewidth=.5)
ax2.plot(data.time, data.auto_corr[n_sweep][4][:], '-g', linewidth=.5)
ax2.plot(data.time, data.auto_corr[n_sweep][5][:], '-b', linewidth=.5)
ax3.plot(data.time, data.auto_corr[n_sweep][6][:], '-r', linewidth=.5)
ax3.plot(data.time, data.auto_corr[n_sweep][7][:], '-g', linewidth=.5)
ax3.plot(data.time, data.auto_corr[n_sweep][8][:], '-b', linewidth=.5)
ax4.plot(data.time, data.auto_corr[n_sweep][9][:], '-r', linewidth=.5)
ax4.plot(data.time, data.auto_corr[n_sweep][10][:], '-g', linewidth=.5)
ax4.plot(data.time, data.auto_corr[n_sweep][11][:], '-b', linewidth=.5)
ax5.plot(data.time, data.auto_corr[n_sweep][12][:], '-r', linewidth=.5)
ax5.plot(data.time, data.auto_corr[n_sweep][13][:], '-g', linewidth=.5)
ax5.plot(data.time, data.auto_corr[n_sweep][14][:], '-b', linewidth=.5)
ax6.plot(data.time, data.auto_corr[n_sweep][15][:], '-r', linewidth=.5)
ax6.plot(data.time, data.auto_corr[n_sweep][16][:], '-g', linewidth=.5)
ax6.plot(data.time, data.auto_corr[n_sweep][17][:], '-b', linewidth=.5)
ax7.plot(data.time, data.auto_corr[n_sweep][18][:], '-r', linewidth=.5)
ax7.plot(data.time, data.auto_corr[n_sweep][19][:], '-g', linewidth=.5)
ax7.plot(data.time, data.auto_corr[n_sweep][20][:], '-b', linewidth=.5)
ax8.plot(data.time, data.auto_corr[n_sweep][21][:], '-r', linewidth=.5)
ax8.plot(data.time, data.auto_corr[n_sweep][22][:], '-g', linewidth=.5)
ax8.plot(data.time, data.auto_corr[n_sweep][23][:], '-b', linewidth=.5)
ax9.plot(data.time, data.auto_corr[n_sweep][24][:], '-r', linewidth=.5)
ax9.plot(data.time, data.auto_corr[n_sweep][25][:], '-g', linewidth=.5)
ax9.plot(data.time, data.auto_corr[n_sweep][26][:], '-b', linewidth=.5)
ax10.plot(data.time, data.auto_corr[n_sweep][27][:], '-r', linewidth=.5)
ax10.plot(data.time, data.auto_corr[n_sweep][28][:], '-g', linewidth=.5)
ax10.plot(data.time, data.auto_corr[n_sweep][29][:], '-b', linewidth=.5)
ax11.plot(data.time, data.auto_corr[n_sweep][30][:], '-r', linewidth=.5)
ax11.plot(data.time, data.auto_corr[n_sweep][31][:], '-g', linewidth=.5)
ax11.plot(data.time, data.auto_corr[n_sweep][32][:], '-b', linewidth=.5)
ax12.plot(data.time, data.auto_corr[n_sweep][33][:], '-r', linewidth=.5)
ax12.plot(data.time, data.auto_corr[n_sweep][34][:], '-g', linewidth=.5)
ax12.plot(data.time, data.auto_corr[n_sweep][35][:], '-b', linewidth=.5)
ax13.plot(data.time, data.auto_corr[n_sweep][36][:], '-r', linewidth=.5)
ax13.plot(data.time, data.auto_corr[n_sweep][37][:], '-g', linewidth=.5)
ax13.plot(data.time, data.auto_corr[n_sweep][38][:], '-b', linewidth=.5)
ax14.plot(data.time, data.auto_corr[n_sweep][39][:], '-r', linewidth=.5)
ax14.plot(data.time, data.auto_corr[n_sweep][40][:], '-g', linewidth=.5)
ax14.plot(data.time, data.auto_corr[n_sweep][41][:], '-b', linewidth=.5)
ax15.plot(data.time, data.auto_corr[n_sweep][42][:], '-r', linewidth=.5)
ax15.plot(data.time, data.auto_corr[n_sweep][43][:], '-g', linewidth=.5)
ax15.plot(data.time, data.auto_corr[n_sweep][44][:], '-b', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][45][:], '-r', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][46][:], '-g', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][47][:], '-gb', linewidth=.5)
auto_corr = data.auto_corr[n_sweep]
auto_corr = np.transpose(auto_corr)
auto_corr1 = auto_corr[0][0:45]
auto_corr2 = auto_corr[22][0:45]
auto_corr3 = auto_corr[44][0:45]
ax16.plot(data.freq, auto_corr1, '-r', linewidth=.5)
ax16.plot(data.freq, auto_corr2, '-g', linewidth=.5)
ax16.plot(data.freq, auto_corr3, '-b', linewidth=.5)

ax1.set_xlabel("1 - 3")
ax2.set_xlabel("4 - 6")
ax3.set_xlabel("7 - 9")
ax4.set_xlabel("10 - 12")
ax5.set_xlabel("13 - 15")
ax6.set_xlabel("16 - 18")
ax7.set_xlabel("19 - 21")
ax8.set_xlabel("22 - 24")
ax9.set_xlabel("25 - 27")
ax10.set_xlabel("28 - 30")
ax11.set_xlabel("31 - 33")
ax12.set_xlabel("34 - 36")
ax13.set_xlabel("37 - 39")
ax14.set_xlabel("40 - 42")
ax15.set_xlabel("43 - 45")
# ax16.set_xlabel("46 - 48")
ax16.set_xlabel("Frequency [kHz]")

# Plot
fig.show
png_fname = work_dir+data_name+'_1st.png'
# fig.savefig(png_fname)

In [ ]:
n_sweep = n_time0

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 4, 1)
ax2 = fig.add_subplot(4, 4, 2)
ax3 = fig.add_subplot(4, 4, 3)
ax4 = fig.add_subplot(4, 4, 4)
ax5 = fig.add_subplot(4, 4, 5)
ax6 = fig.add_subplot(4, 4, 6)
ax7 = fig.add_subplot(4, 4, 7)
ax8 = fig.add_subplot(4, 4, 8)
ax9 = fig.add_subplot(4, 4, 9)
ax10 = fig.add_subplot(4, 4, 10)
ax11 = fig.add_subplot(4, 4, 11)
ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13)
ax14 = fig.add_subplot(4, 4, 14)
ax15 = fig.add_subplot(4, 4, 15)
ax16 = fig.add_subplot(4, 4, 16)

ax1.plot(data.time, data.auto_corr[n_sweep][0][:], '-r', linewidth=.5)
ax1.plot(data.time, data.auto_corr[n_sweep][1][:], '-g', linewidth=.5)
ax1.plot(data.time, data.auto_corr[n_sweep][2][:], '-b', linewidth=.5)
ax2.plot(data.time, data.auto_corr[n_sweep][3][:], '-r', linewidth=.5)
ax2.plot(data.time, data.auto_corr[n_sweep][4][:], '-g', linewidth=.5)
ax2.plot(data.time, data.auto_corr[n_sweep][5][:], '-b', linewidth=.5)
ax3.plot(data.time, data.auto_corr[n_sweep][6][:], '-r', linewidth=.5)
ax3.plot(data.time, data.auto_corr[n_sweep][7][:], '-g', linewidth=.5)
ax3.plot(data.time, data.auto_corr[n_sweep][8][:], '-b', linewidth=.5)
ax4.plot(data.time, data.auto_corr[n_sweep][9][:], '-r', linewidth=.5)
ax4.plot(data.time, data.auto_corr[n_sweep][10][:], '-g', linewidth=.5)
ax4.plot(data.time, data.auto_corr[n_sweep][11][:], '-b', linewidth=.5)
ax5.plot(data.time, data.auto_corr[n_sweep][12][:], '-r', linewidth=.5)
ax5.plot(data.time, data.auto_corr[n_sweep][13][:], '-g', linewidth=.5)
ax5.plot(data.time, data.auto_corr[n_sweep][14][:], '-b', linewidth=.5)
ax6.plot(data.time, data.auto_corr[n_sweep][15][:], '-r', linewidth=.5)
ax6.plot(data.time, data.auto_corr[n_sweep][16][:], '-g', linewidth=.5)
ax6.plot(data.time, data.auto_corr[n_sweep][17][:], '-b', linewidth=.5)
ax7.plot(data.time, data.auto_corr[n_sweep][18][:], '-r', linewidth=.5)
ax7.plot(data.time, data.auto_corr[n_sweep][19][:], '-g', linewidth=.5)
ax7.plot(data.time, data.auto_corr[n_sweep][20][:], '-b', linewidth=.5)
ax8.plot(data.time, data.auto_corr[n_sweep][21][:], '-r', linewidth=.5)
ax8.plot(data.time, data.auto_corr[n_sweep][22][:], '-g', linewidth=.5)
ax8.plot(data.time, data.auto_corr[n_sweep][23][:], '-b', linewidth=.5)
ax9.plot(data.time, data.auto_corr[n_sweep][24][:], '-r', linewidth=.5)
ax9.plot(data.time, data.auto_corr[n_sweep][25][:], '-g', linewidth=.5)
ax9.plot(data.time, data.auto_corr[n_sweep][26][:], '-b', linewidth=.5)
ax10.plot(data.time, data.auto_corr[n_sweep][27][:], '-r', linewidth=.5)
ax10.plot(data.time, data.auto_corr[n_sweep][28][:], '-g', linewidth=.5)
ax10.plot(data.time, data.auto_corr[n_sweep][29][:], '-b', linewidth=.5)
ax11.plot(data.time, data.auto_corr[n_sweep][30][:], '-r', linewidth=.5)
ax11.plot(data.time, data.auto_corr[n_sweep][31][:], '-g', linewidth=.5)
ax11.plot(data.time, data.auto_corr[n_sweep][32][:], '-b', linewidth=.5)
ax12.plot(data.time, data.auto_corr[n_sweep][33][:], '-r', linewidth=.5)
ax12.plot(data.time, data.auto_corr[n_sweep][34][:], '-g', linewidth=.5)
ax12.plot(data.time, data.auto_corr[n_sweep][35][:], '-b', linewidth=.5)
ax13.plot(data.time, data.auto_corr[n_sweep][36][:], '-r', linewidth=.5)
ax13.plot(data.time, data.auto_corr[n_sweep][37][:], '-g', linewidth=.5)
ax13.plot(data.time, data.auto_corr[n_sweep][38][:], '-b', linewidth=.5)
ax14.plot(data.time, data.auto_corr[n_sweep][39][:], '-r', linewidth=.5)
ax14.plot(data.time, data.auto_corr[n_sweep][40][:], '-g', linewidth=.5)
ax14.plot(data.time, data.auto_corr[n_sweep][41][:], '-b', linewidth=.5)
ax15.plot(data.time, data.auto_corr[n_sweep][42][:], '-r', linewidth=.5)
ax15.plot(data.time, data.auto_corr[n_sweep][43][:], '-g', linewidth=.5)
ax15.plot(data.time, data.auto_corr[n_sweep][44][:], '-b', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][45][:], '-r', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][46][:], '-g', linewidth=.5)
# ax16.plot(data.time, data.auto_corr[n_sweep][47][:], '-gb', linewidth=.5)
auto_corr = data.auto_corr[n_sweep]
auto_corr = np.transpose(auto_corr)
auto_corr1 = auto_corr[0][0:45]
auto_corr2 = auto_corr[22][0:45]
auto_corr3 = auto_corr[44][0:45]
ax16.plot(data.freq, auto_corr1, '-r', linewidth=.5)
ax16.plot(data.freq, auto_corr2, '-g', linewidth=.5)
ax16.plot(data.freq, auto_corr3, '-b', linewidth=.5)

ax1.set_xlabel("1 - 3")
ax2.set_xlabel("4 - 6")
ax3.set_xlabel("7 - 9")
ax4.set_xlabel("10 - 12")
ax5.set_xlabel("13 - 15")
ax6.set_xlabel("16 - 18")
ax7.set_xlabel("19 - 21")
ax8.set_xlabel("22 - 24")
ax9.set_xlabel("25 - 27")
ax10.set_xlabel("28 - 30")
ax11.set_xlabel("31 - 33")
ax12.set_xlabel("34 - 36")
ax13.set_xlabel("37 - 39")
ax14.set_xlabel("40 - 42")
ax15.set_xlabel("43 - 45")
# ax16.set_xlabel("46 - 48")
ax16.set_xlabel("Frequency [kHz]")

# Plot
fig.show
png_fname = work_dir+data_name+'_last.png'
# fig.savefig(png_fname)